In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math
from numba import jit
from pylab import imshow, show
from timeit import default_timer as timer

In [ ]:
def mandel(x, y, max_iters):
  """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
  """
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 4:
      return i

  return max_iters
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height
    
  for x in range(width):
    real = min_x + x * pixel_size_x
    for y in range(height):
      imag = min_y + y * pixel_size_y
      color = mandel(real, imag, iters)
      image[y, x] = color
      
image = np.zeros((1024, 1536), dtype = np.uint8)
start = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20) 
dt = timer() - start

print("Mandelbrot created in %f s" % dt)
imshow(image)
show()

In [ ]:
@jit(target_backend='gpu')
def mandel(x, y, max_iters):
  """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
  """
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 4:
      return i

  return max_iters

@jit(target_backend='gpu')
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height
    
  for x in range(width):
    real = min_x + x * pixel_size_x
    for y in range(height):
      imag = min_y + y * pixel_size_y
      color = mandel(real, imag, iters)
      image[y, x] = color
      
image = np.zeros((1024, 1536), dtype = np.uint8)
start = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20) 
dt = timer() - start

print("Mandelbrot created in %f s" % dt)
imshow(image)
show()

In [ ]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize

@vectorize(['float32(float32,float32)'])

def multiplyVectors(a,b):
  return a*b

def main():
  n = 64000000
  A = np.ones(n,dtype=np.float32)
  B = np.ones(n,dtype=np.float32)
  C = np.ones(n,dtype=np.float32)
  start = timer()
  multiplyVectors(A,B,C)
  vectorMultiplyTimer = timer() - start
  print("C[:3] = " + str(C[:3]))
  print("C[:-3] = " + str(C[:-3]))
  print("It took %f seconds" % vectorMultiplyTimer)

main()

In [ ]:
cpuTime = 22.699887 # seconds
vectorizeTime = 0.050464 # seconds
print("Speed improvement of %f times faster" % round(cpuTime/vectorizeTime,2))

In [ ]:
from numba import cuda, jit
n = 1000000
def fillArrayWithoutGPU(a):
  for i in range(n):
    a[i] += 1
@jit(target_backend='cuda')
def fillArrayWithGPU(a):
  for i in range(n):
    a[i] += 1

def main():
  arrayToFill = np.zeros(n)
  start = timer()
  fillArrayWithoutGPU(arrayToFill)
  endWithoutGPU = timer() - start
  arrayToFill = np.zeros(n) 
  startGPU = timer()
  fillArrayWithGPU(arrayToFill)
  endWithGPU = timer() - startGPU
  print("Without the GPU it took %f seconds" % endWithoutGPU)
  print("With the GPU %f seconds" % endWithGPU)

main()